## 기본 예시: 프롬프트 + 모델 + 출력 파서

가장 기본적이고 일반적인 사용 사례는 prompt 템플릿과 모델을 함께 연결하는 것입니다. 이것이 어떻게 작동하는지 보기 위해, 각 나라별 수도를 물어보는 Chain을 생성해 보겠습니다.


In [1]:
# API KEY를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API KEY 정보로드
load_dotenv()

True

In [2]:
# LangSmith 추적을 설정합니다. https://smith.langchain.com
# !pip install -qU langchain-teddynote
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("CH01-Basic")

LangSmith 추적을 시작합니다.
[프로젝트명]
CH01-Basic


## 프롬프트 템플릿의 활용

`PromptTemplate`

- 사용자의 입력 변수를 사용하여 완전한 프롬프트 문자열을 만드는 데 사용되는 템플릿입니다
- 사용법
  - `template`: 템플릿 문자열입니다. 이 문자열 내에서 중괄호 `{}`는 변수를 나타냅니다.
  - `input_variables`: 중괄호 안에 들어갈 변수의 이름을 리스트로 정의합니다.

`input_variables`

- input_variables는 PromptTemplate에서 사용되는 변수의 이름을 정의하는 리스트입니다.

In [3]:
from langchain_teddynote.messages import stream_response  # 스트리밍 출력
from langchain_core.prompts import PromptTemplate

`from_template()` 메소드를 사용하여 PromptTemplate 객체 생성


In [4]:
# template 정의
template = "{country}의 수도는 어디인가요?"

# from_template 메소드를 이용하여 PromptTemplate 객체 생성
prompt_template = PromptTemplate.from_template(template)
prompt_template

PromptTemplate(input_variables=['country'], input_types={}, partial_variables={}, template='{country}의 수도는 어디인가요?')

In [5]:
# prompt 생성
prompt = prompt_template.format(country="대한민국")
prompt

'대한민국의 수도는 어디인가요?'

In [6]:
# prompt 생성
prompt = prompt_template.format(country="미국")
prompt

'미국의 수도는 어디인가요?'

In [7]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0.1,
)

## Chain 생성

### LCEL(LangChain Expression Language)

![lcel.png](./images/lcel.png)

여기서 우리는 LCEL을 사용하여 다양한 구성 요소를 단일 체인으로 결합합니다

```
chain = prompt | model | output_parser
```

`|` 기호는 [unix 파이프 연산자](<https://en.wikipedia.org/wiki/Pipeline_(Unix)>)와 유사하며, 서로 다른 구성 요소를 연결하고 한 구성 요소의 출력을 다음 구성 요소의 입력으로 전달합니다.

이 체인에서 사용자 입력은 프롬프트 템플릿으로 전달되고, 그런 다음 프롬프트 템플릿 출력은 모델로 전달됩니다. 각 구성 요소를 개별적으로 살펴보면 무슨 일이 일어나고 있는지 이해할 수 있습니다.


In [8]:
# prompt 를 PromptTemplate 객체로 생성합니다.
prompt = PromptTemplate.from_template("{topic} 에 대해 쉽게 설명해주세요.")

model = ChatOpenAI(model="gpt-4o-mini", temperature=0.1)

chain = prompt | model

### invoke() 호출

- python 딕셔너리 형태로 입력값을 전달합니다.(키: 값)
- invoke() 함수 호출 시, 입력값을 전달합니다.

In [9]:
# input 딕셔너리에 주제를 '인공지능 모델의 학습 원리'으로 설정합니다.
input = {"topic": "인공지능 모델의 학습 원리"}

In [11]:
# prompt 객체와 model 객체를 파이프(|) 연산자로 연결하고 invoke 메서드를 사용하여 input을 전달합니다.
# 이를 통해 AI 모델이 생성한 메시지를 반환합니다.
chain.invoke(input)

AIMessage(content='인공지능 모델의 학습 원리를 쉽게 설명하자면, 다음과 같은 단계로 이해할 수 있습니다.\n\n1. **데이터 수집**: 인공지능 모델은 학습하기 위해 많은 데이터를 필요로 합니다. 이 데이터는 이미지, 텍스트, 소리 등 다양한 형태일 수 있습니다.\n\n2. **데이터 전처리**: 수집한 데이터는 모델이 이해할 수 있는 형태로 가공해야 합니다. 예를 들어, 이미지의 크기를 조정하거나, 텍스트를 숫자로 변환하는 과정이 필요합니다.\n\n3. **모델 선택**: 학습할 모델을 선택합니다. 이는 신경망, 결정 트리, 서포트 벡터 머신 등 여러 종류가 있습니다. 각 모델은 특정한 문제에 더 적합할 수 있습니다.\n\n4. **학습**: 모델은 주어진 데이터를 바탕으로 패턴을 학습합니다. 이 과정에서 모델은 입력 데이터와 정답(레이블) 간의 관계를 찾아내려고 합니다. 예를 들어, 고양이와 개의 이미지를 구분하는 모델은 각각의 특징을 학습하게 됩니다.\n\n5. **손실 함수**: 모델의 예측이 실제 정답과 얼마나 차이가 나는지를 측정하는 손실 함수를 사용합니다. 이 값을 최소화하는 방향으로 모델의 파라미터(가중치)를 조정합니다.\n\n6. **최적화**: 경량화된 알고리즘(예: 경사 하강법)을 사용하여 손실 함수를 최소화하는 방향으로 모델의 파라미터를 업데이트합니다. 이 과정을 여러 번 반복하여 모델이 점점 더 정확해지도록 합니다.\n\n7. **검증**: 학습이 끝난 후, 모델의 성능을 평가하기 위해 별도의 검증 데이터를 사용합니다. 이 데이터는 모델이 학습할 때 사용하지 않은 데이터로, 모델의 일반화 능력을 확인하는 데 도움을 줍니다.\n\n8. **배포 및 사용**: 모델이 충분히 학습하고 검증되면 실제 환경에 배포하여 사용합니다. 이후에도 새로운 데이터로 모델을 업데이트하거나 재학습할 수 있습니다.\n\n이러한 과정을 통해 인공지능 모델은 주어진 문제를 해결할 수 있는 능력을 갖추게 됩니다.', additional_kwargs=

아래는 스트리밍을 출력하는 예시 입니다.

In [12]:
# 스트리밍 출력을 위한 요청
answer = chain.stream(input)
# 스트리밍 출력
stream_response(answer)

인공지능 모델의 학습 원리를 쉽게 설명하자면, 다음과 같은 단계로 나눌 수 있습니다.

1. **데이터 수집**: 인공지능 모델은 학습하기 위해 많은 데이터를 필요로 합니다. 예를 들어, 고양이와 개를 구분하는 모델을 만들고 싶다면, 고양이와 개의 사진이 포함된 데이터셋이 필요합니다.

2. **데이터 전처리**: 수집한 데이터는 종종 정리하고 변환해야 합니다. 예를 들어, 이미지의 크기를 맞추거나, 텍스트를 숫자로 변환하는 등의 작업이 필요합니다.

3. **모델 선택**: 학습할 모델을 선택합니다. 이는 신경망, 결정 트리, 서포트 벡터 머신 등 다양한 형태가 있을 수 있습니다. 각 모델은 특정한 문제에 더 적합할 수 있습니다.

4. **학습 과정**: 모델은 주어진 데이터를 통해 패턴을 학습합니다. 이 과정에서 모델은 입력 데이터(예: 이미지)와 정답(예: 고양이 또는 개)을 비교하여, 얼마나 잘 예측했는지를 평가합니다. 이 평가 결과를 바탕으로 모델의 내부 파라미터를 조정하여 점점 더 정확한 예측을 할 수 있도록 합니다.

5. **손실 함수**: 모델의 예측이 실제 정답과 얼마나 차이가 나는지를 측정하는 방법입니다. 이 값을 최소화하는 방향으로 모델을 학습시킵니다.

6. **검증 및 테스트**: 학습이 끝난 후, 모델의 성능을 평가하기 위해 새로운 데이터(훈련에 사용되지 않은 데이터)를 사용합니다. 이를 통해 모델이 실제 상황에서도 잘 작동하는지를 확인합니다.

7. **배포 및 활용**: 모델이 충분히 잘 학습되었다면, 실제 애플리케이션에 배포하여 사용합니다. 예를 들어, 사진을 올리면 고양이인지 개인지 자동으로 분류해주는 앱에 적용할 수 있습니다.

이러한 과정을 통해 인공지능 모델은 데이터를 기반으로 학습하고, 새로운 상황에서도 유용한 예측을 할 수 있게 됩니다.

### 출력파서(Output Parser)


In [13]:
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

Chain 에 출력파서를 추가합니다.

In [14]:
# 프롬프트, 모델, 출력 파서를 연결하여 처리 체인을 구성합니다.
chain = prompt | model | output_parser

In [15]:
# chain 객체의 invoke 메서드를 사용하여 input을 전달합니다.
input = {"topic": "인공지능 모델의 학습 원리"}
chain.invoke(input)

'인공지능 모델의 학습 원리를 쉽게 설명하자면, 다음과 같은 단계로 나눌 수 있습니다.\n\n1. **데이터 수집**: 인공지능 모델은 학습하기 위해 많은 데이터를 필요로 합니다. 이 데이터는 이미지, 텍스트, 소리 등 다양한 형태일 수 있습니다.\n\n2. **데이터 전처리**: 수집한 데이터는 종종 정리하고 변환해야 합니다. 예를 들어, 이미지의 크기를 맞추거나, 텍스트를 숫자로 변환하는 등의 작업이 필요합니다.\n\n3. **모델 선택**: 학습할 모델을 선택합니다. 모델은 데이터를 처리하고 패턴을 학습하는 알고리즘입니다. 예를 들어, 신경망, 결정 트리, 서포트 벡터 머신 등이 있습니다.\n\n4. **학습 과정**: 모델은 주어진 데이터를 바탕으로 학습을 시작합니다. 이 과정에서 모델은 입력 데이터와 정답(라벨)을 비교하여 오차를 계산합니다. 이 오차를 줄이기 위해 모델의 파라미터(가중치)를 조정합니다. 이 과정을 반복하면서 모델은 점점 더 정확한 예측을 할 수 있게 됩니다.\n\n5. **검증**: 학습이 끝난 후, 모델의 성능을 평가하기 위해 새로운 데이터를 사용합니다. 이 데이터는 모델이 학습할 때 사용하지 않았던 데이터로, 모델이 얼마나 잘 일반화되었는지를 확인하는 데 도움을 줍니다.\n\n6. **조정 및 개선**: 모델의 성능이 만족스럽지 않다면, 하이퍼파라미터를 조정하거나 더 많은 데이터를 수집하여 다시 학습할 수 있습니다.\n\n7. **배포**: 최종적으로 학습된 모델은 실제 환경에서 사용될 수 있도록 배포됩니다. 사용자는 이 모델을 통해 예측이나 분류 등의 작업을 수행할 수 있습니다.\n\n이러한 과정을 통해 인공지능 모델은 데이터를 기반으로 학습하고, 새로운 상황에서도 유용한 정보를 제공할 수 있게 됩니다.'

In [16]:
# 스트리밍 출력을 위한 요청
answer = chain.stream(input)
# 스트리밍 출력
stream_response(answer)

인공지능 모델의 학습 원리를 쉽게 설명하자면, 다음과 같은 단계로 나눌 수 있습니다.

1. **데이터 수집**: 인공지능 모델은 학습하기 위해 많은 데이터를 필요로 합니다. 이 데이터는 이미지, 텍스트, 소리 등 다양한 형태일 수 있습니다.

2. **데이터 전처리**: 수집한 데이터는 종종 정리하고 변환해야 합니다. 예를 들어, 이미지의 크기를 맞추거나, 텍스트를 숫자로 변환하는 등의 작업이 필요합니다.

3. **모델 선택**: 학습할 모델을 선택합니다. 이는 신경망, 결정 트리, 서포트 벡터 머신 등 여러 종류가 있습니다. 각 모델은 특정한 문제에 더 적합할 수 있습니다.

4. **학습**: 모델은 데이터를 통해 패턴을 학습합니다. 이 과정에서 모델은 입력 데이터와 정답(라벨)을 비교하여 오차를 계산하고, 이 오차를 줄이기 위해 모델의 파라미터(가중치)를 조정합니다. 이 과정을 반복하면서 모델은 점점 더 정확해집니다.

5. **검증**: 학습이 끝난 후, 모델의 성능을 평가하기 위해 새로운 데이터(검증 데이터)를 사용합니다. 이 데이터는 모델이 학습할 때 사용하지 않았던 데이터로, 모델이 얼마나 잘 일반화되었는지를 확인하는 데 도움을 줍니다.

6. **조정 및 개선**: 모델의 성능이 만족스럽지 않다면, 하이퍼파라미터를 조정하거나, 더 많은 데이터를 수집하거나, 다른 모델을 시도하는 등의 방법으로 개선할 수 있습니다.

7. **배포**: 최종적으로 학습된 모델은 실제 환경에 배포되어 사용됩니다. 사용자가 입력한 데이터를 바탕으로 예측을 하거나, 분류를 수행하는 등의 작업을 하게 됩니다.

이러한 과정을 통해 인공지능 모델은 주어진 문제를 해결할 수 있는 능력을 갖추게 됩니다.

### 템플릿을 변경하여 적용

- 아래의 프롬프트 내용을 얼마든지 **변경** 하여 테스트 해볼 수 있습니다.
- `model_name` 역시 변경하여 테스트가 가능합니다.

In [17]:
template = """
당신은 영어를 가르치는 10년차 영어 선생님입니다. 주어진 상황에 맞는 영어 회화를 작성해 주세요.
양식은 [FORMAT]을 참고하여 작성해 주세요.

#상황:
{question}

#FORMAT:
- 영어 회화:
- 한글 해석:
"""

# 프롬프트 템플릿을 이용하여 프롬프트를 생성합니다.
prompt = PromptTemplate.from_template(template)

# ChatOpenAI 챗모델을 초기화합니다.
model = ChatOpenAI(model_name="gpt-4o-mini")

# 문자열 출력 파서를 초기화합니다.
output_parser = StrOutputParser()

In [18]:
# 체인을 구성합니다.
chain = prompt | model | output_parser

In [19]:
# 완성된 Chain을 실행하여 답변을 얻습니다.
print(chain.invoke({"question": "저는 식당에 가서 음식을 주문하고 싶어요"}))

- 영어 회화:  
  **Customer:** Hi, I’d like to see the menu, please.  
  **Waiter:** Of course! Here you go. Can I get you something to drink while you look?  
  **Customer:** Yes, I’d like a glass of water, please.  
  **Waiter:** Sure! Are you ready to order, or do you need a bit more time?  
  **Customer:** I think I’m ready. I’ll have the grilled chicken salad, please.  
  **Waiter:** Great choice! Would you like any dressing with that?  
  **Customer:** Yes, please. I’d like balsamic vinaigrette.  
  **Waiter:** Perfect! I’ll get that for you right away.  

- 한글 해석:  
  **고객:** 안녕하세요, 메뉴를 보고 싶어요.  
  **웨이터:** 물론이죠! 여기 있습니다. 보시면서 음료수 주문할까요?  
  **고객:** 네, 물 한 잔 주세요.  
  **웨이터:** 알겠습니다! 주문할 준비가 되셨나요, 아니면 좀 더 필요하신가요?  
  **고객:** 저는 준비가 된 것 같아요. 그릴에 구운 치킨 샐러드 하나 주세요.  
  **웨이터:** 좋은 선택입니다! 드레싱은 어떤 걸 원하시나요?  
  **고객:** 네, 발사믹 비네그레트를 주세요.  
  **웨이터:** 완벽해요! 바로 가져다 드릴게요.  


In [20]:
# 완성된 Chain을 실행하여 답변을 얻습니다.
# 스트리밍 출력을 위한 요청
answer = chain.stream({"question": "저는 식당에 가서 음식을 주문하고 싶어요"})
# 스트리밍 출력
stream_response(answer)

- 영어 회화:  
  **Waiter:** Good evening! Welcome to our restaurant. How many people are in your party?  
  **You:** Good evening! It's just me tonight.  
  **Waiter:** Great! Here’s the menu. Can I start you off with something to drink?  
  **You:** Yes, I’d like a glass of water, please.  
  **Waiter:** Sure! Are you ready to order your food?  
  **You:** Yes, I’d like the grilled chicken with vegetables, please.  
  **Waiter:** Excellent choice! Would you like any sides with that?  
  **You:** Yes, can I have a side of mashed potatoes, please?  
  **Waiter:** Of course! I’ll get that order in for you.  
  **You:** Thank you!

- 한글 해석:  
  **웨이터:** 좋은 저녁입니다! 저희 식당에 오신 것을 환영합니다. 몇 분이신가요?  
  **당신:** 좋은 저녁입니다! 오늘은 저 혼자입니다.  
  **웨이터:** 좋습니다! 여기 메뉴입니다. 음료를 먼저 주문하시겠어요?  
  **당신:** 네, 물 한 잔 주세요.  
  **웨이터:** 알겠습니다! 음식을 주문할 준비 되셨나요?  
  **당신:** 네, 구운 치킨과 야채를 주문하고 싶어요.  
  **웨이터:** 훌륭한 선택입니다! 사이드 메뉴는 필요하신가요?  
  **당신:** 네, 으깬 감자를 사이드로 주세요.  
  **웨이터:** 물론입니다! 주문 넣어드릴게요.  
  **당신:** 감사합니다!

In [21]:
# 이번에는 question 을 '미국에서 피자 주문'으로 설정하여 실행합니다.
# 스트리밍 출력을 위한 요청
answer = chain.stream({"question": "미국에서 피자 주문"})
# 스트리밍 출력
stream_response(answer)

- 영어 회화:
Customer: Hi, I’d like to place an order for a pizza, please.  
Pizza Shop: Sure! What size would you like?  
Customer: I’d like a large pizza.  
Pizza Shop: Great! What toppings do you want?  
Customer: Can I have pepperoni and mushrooms, please?  
Pizza Shop: Absolutely! Would you like anything else?  
Customer: Yes, can I also get a side of garlic bread?  
Pizza Shop: Of course! Your total comes to $25. Would you like to pay with cash or card?  
Customer: I’ll pay with a card.  
Pizza Shop: Perfect! Your order will be ready in about 30 minutes.  
Customer: Thank you!  

- 한글 해석:
고객: 안녕하세요, 피자 주문하고 싶어요.  
피자 가게: 물론이죠! 어떤 사이즈로 드릴까요?  
고객: 대형 피자로 주세요.  
피자 가게: 좋습니다! 어떤 토핑을 원하시나요?  
고객: 페퍼로니와 버섯으로 부탁해요.  
피자 가게: 알겠습니다! 다른 것도 필요하신가요?  
고객: 네, 마늘빵도 하나 추가해 주세요.  
피자 가게: 물론입니다! 총 금액은 25달러입니다. 현금으로 결제하시겠습니까, 아니면 카드로 하시겠습니까?  
고객: 카드로 결제할게요.  
피자 가게: 완벽합니다! 주문은 약 30분 후에 준비될 것입니다.  
고객: 감사합니다!